<h5>This is done in <span style="color:blue">Google colab</span>. If you are doing this in local machine then don't forget to install necessary modules like tenforflow hub, pandas, numpy, tensorflow-text,etc</h5>

In [ ]:
!pip install tensorflow-text

In [3]:
import pandas as pd
import numpy as np
import tensorflow_hub as hub
import tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
df=pd.read_csv("./food_delivery_customer_service.csv")

In [1]:
df.head(3)['Message'][2]


I would like to cancel the order done for 1 plate chicken momo


In [ ]:
df_input=df[df['Category']]
df_input.shape

In [ ]:
df_output=df[df['sentiment']]

In [10]:
df_balanced=pd.concat([df_input, df_output])

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
x_train,x_test,y_train,y_test=train_test_split(df_input,df_output,train_test_split=0.8)

In [19]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
# https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [20]:
def get_sentence_embedding(sentences):
  preprocessed_text=bert_preprocess(sentences)
  return bert_encoder(preprocessed_text)["pooled_output"]

get_sentence_embedding(["500$ discount. hurry up","Dilip you have a volleyball game tomorrow"])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.84351724, -0.51327276, -0.88845724, ..., -0.74748844,
        -0.7531474 ,  0.91964495],
       [-0.77610546, -0.551759  , -0.75852644, ..., -0.49820265,
        -0.7498885 ,  0.82108516]], dtype=float32)>

In [24]:
import tensorflow as tf

In [25]:
# Bert Layers
text_input=tf.keras.layers.Input(shape=(),dtype=tf.string,name="text")
preprocessed_text=bert_preprocess(text_input)
outputs=bert_encoder(preprocessed_text)

# Neural network layers
l1=tf.keras.layers.Dropout(0.1,name="dropout")(outputs['pooled_output'])
l2=tf.keras.layers.Dense(3,activation='softmax',name="output")(l1)

# construct the final layers
model=tf.keras.Model(inputs=[text_input],outputs=[l2])

In [26]:
Metrics=[
         tf.keras.metrics.BinaryAccuracy(name="accuracy"),
         tf.keras.metrics.Precision(name="precision"),
         tf.keras.metrics.Recall(name="recall")
]

model.compile(optimizer='adam',
              loss="categorical_crossentropy",
              metrics=Metrics)

model.fit(x_train,y_train,epochs=10)

Epoch 1/10
35/35 [==============================] - 27s 328ms/step - loss: 0.6235 - accuracy: 0.6795 - precision: 0.6922 - recall: 0.6464
Epoch 2/10
35/35 [==============================] - 11s 306ms/step - loss: 0.5001 - accuracy: 0.8241 - precision: 0.8168 - recall: 0.8357
Epoch 3/10
35/35 [==============================] - 11s 307ms/step - loss: 0.4234 - accuracy: 0.8643 - precision: 0.8517 - recall: 0.8821
Epoch 4/10
35/35 [==============================] - 11s 308ms/step - loss: 0.3708 - accuracy: 0.8884 - precision: 0.8822 - recall: 0.8964
Epoch 5/10
35/35 [==============================] - 11s 317ms/step - loss: 0.3438 - accuracy: 0.8893 - precision: 0.8838 - recall: 0.8964
Epoch 6/10
35/35 [==============================] - 11s 314ms/step - loss: 0.3169 - accuracy: 0.9027 - precision: 0.8881 - recall: 0.9214
Epoch 7/10
35/35 [==============================] - 11s 321ms/step - loss: 0.3009 - accuracy: 0.9062 - precision: 0.8998 - recall: 0.9143
Epoch 8/10
35/35 [================

In [27]:
model.evaluate(x_test,y_test)

12/12 [==============================] - 7s 369ms/step - loss: 0.3457 - accuracy: 0.8824 - precision: 0.8705 - recall: 0.8984


[0.3457362651824951,
 0.8823529481887817,
 0.8704662919044495,
 0.8983957171440125]

In [38]:
y_predicted=model.predict(x_test)

12/12 [==============================] - 5s 348ms/step
